# youtube dlp example
## make sure to upload the yt_cookies.txt in the content folder for the run time!
### !yt-dlp -x --audio-format mp3 --audio-quality 0 --no-overwrites --continue --sleep-interval 5 -o output_location URL


In [ ]:
!pip install yt-dlp

In [ ]:
import os

def count_audio_files(directory):
    """Counts the number of audio files in a directory and its subdirectories."""
    audio_extensions = ['.mp3', '.wav', '.ogg', '.flac', '.aac', '.m4a'] # Add or remove extensions as needed
    count = 0
    for root, dirs, files in os.walk(directory):
        for file in files:
            if any(file.lower().endswith(ext) for ext in audio_extensions):
                count += 1
    return count

audio_data_path = '/content/drive/MyDrive/CQTNet/crawl_data/CQTNet_data'
total_audio_tracks = count_audio_files(audio_data_path)

print(f"Total number of audio tracks in {audio_data_path}: {total_audio_tracks}")

Total number of audio tracks in /content/drive/MyDrive/CQTNet/crawl_data/CQTNet_data: 232


# mount google drive to my personal drive so i can store audio tracks between sessions

In [ ]:
from google.colab import drive
drive.mount('/content/drive')  # follow prompt

import os

DRIVE_BASE = '/content/drive/MyDrive/CQTNet'  # persistent project folder
AUDIO_DIR = os.path.join(DRIVE_BASE, 'crawl_data', 'data')
MANIFEST = os.path.join(DRIVE_BASE, 'download_manifest_old.csv')

os.makedirs(AUDIO_DIR, exist_ok=True)
os.makedirs(os.path.dirname(MANIFEST), exist_ok=True)

print("Audio will be stored in:", AUDIO_DIR)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Audio will be stored in: /content/drive/MyDrive/CQTNet/crawl_data/data


In [ ]:
!yt-dlp --cookies /content/yt_cookies2.txt -f bestaudio --no-overwrites --continue --download-sections "*00:00:00-00:30" --sleep-interval 5 --extract-audio --audio-format mp3 --audio-quality 0 -o "{AUDIO_DIR}/{set_id}/{ver_id}.%(ext)s" https://www.youtube.com/watch?v=MJr2we19F7E

[youtube] Extracting URL: https://www.youtube.com/watch?v=MJr2we19F7E
[youtube] MJr2we19F7E: Downloading webpage
[youtube] MJr2we19F7E: Downloading tv client config
[youtube] MJr2we19F7E: Downloading tv player API JSON
[youtube] MJr2we19F7E: Downloading web safari player API JSON
[youtube] MJr2we19F7E: Downloading player c6d7bdc9-main
         player = https://www.youtube.com/s/player/c6d7bdc9/player_ias.vflset/en_US/base.js
         n = O5gjJnnOFJcQGBRr ; player = https://www.youtube.com/s/player/c6d7bdc9/player_ias.vflset/en_US/base.js
         Please report this issue on  https://github.com/yt-dlp/yt-dlp/issues?q= , filling out the appropriate issue template. Confirm you are on the latest version using  yt-dlp -U
         n = C1-AaLrsuGtx2C5Y ; player = https://www.youtube.com/s/player/c6d7bdc9/player_ias.vflset/en_US/base.js
         Please report this issue on  https://github.com/yt-dlp/yt-dlp/issues?q= , filling out the appropriate issue template. Confirm you are on the latest ve

# load shs100k metadata and compute subset
## validation and test sets must have a version contained in the test set
## displayed number of tracks already downloaded

### i modified this cell several times. initially i tried to copy the training, vlaidation, and test splits from the SHS100k repo but obviously that's way too many audio files. So after downloading a few hundred tracks, I filled out the rest of the training splits adding on to what I downloaded. It's necessary that all tracks in the test set have a version be seen in the training data so it can have the ability to classify it.

In [ ]:
import pandas as pd, math, csv, os

LIST_PATH  = '/content/list'
VAL_PATH = '/content/val'
TEST_PATH = '/content/test'
TRAIN_PATH = '/content/train'

# Use the specified download_manifest.csv as the source for initial training tracks and manifest
AUDIO_DIR = '/content/drive/MyDrive/CQTNet/crawl_data/data'
MANIFEST = os.path.join(AUDIO_DIR, 'download_manifest.csv') # This will be the manifest for resuming

os.makedirs(AUDIO_DIR, exist_ok=True)
print("Audio will be stored in:", AUDIO_DIR)
print("Manifest file for resuming:", MANIFEST)

#i had multiple version of the manifest file which i concatenated into one but they had differing column formats so i accept both.
list_df = pd.read_csv(LIST_PATH, sep='\t', header=None, engine='python', quoting=csv.QUOTE_NONE)
if list_df.shape[1] < 6:
    if list_df.shape[1] >= 5:
         list_df.columns = ['set_id','ver_id','title','performer','url'][:list_df.shape[1]]
    else:
        if list_df.shape[1] >= 4:
             list_df.columns = ['set_id','ver_id','title','performer'][:list_df.shape[1]]
        else:
            raise ValueError(f"The list file at {LIST_PATH} does not contain enough columns. Expected at least 4, found {list_df.shape[1]}.")
else:
    list_df = list_df.iloc[:, :6]
    list_df.columns = ['set_id','ver_id','title','performer','url','status']

list_df = list_df.astype({'set_id':int,'ver_id':int})

#load original splits to work off of
def load_split(path):
    return pd.read_csv(path, sep='\t', header=None, names=['set_id','ver_id'],
                       engine='python', quoting=csv.QUOTE_NONE).astype(int)

original_train_df = load_split(TRAIN_PATH)
original_val_df   = load_split(VAL_PATH)
original_test_df  = load_split(TEST_PATH)


#load current manifest to define what has been already downloaded
initial_train_df = pd.DataFrame(columns=['set_id', 'ver_id'])
existing = {} # Initialize existing dictionary here

if os.path.exists(MANIFEST):
    try:
        # Use csv module to read the manifest robustly
        manifest_data = []
        with open(MANIFEST, 'r', newline='') as f:
            reader = csv.reader(f)
            # Assume the first row is the header, but handle missing headers
            try:
                header = next(reader)
                # Try to find set_id and ver_id columns, case-insensitive
                header_lower = [h.lower() for h in header]
                try:
                    set_id_idx = header_lower.index('set_id')
                    ver_id_idx = header_lower.index('ver_id')
                    # If status column exists, get its index
                    try:
                        status_idx = header_lower.index('status')
                    except ValueError:
                        status_idx = -1 # Indicate no status column

                    for row in reader:
                        if len(row) > max(set_id_idx, ver_id_idx): # Ensure row has enough fields for set_id and ver_id
                            try:
                                set_id = int(row[set_id_idx])
                                ver_id = int(row[ver_id_idx])
                                status = row[status_idx] if status_idx != -1 and status_idx < len(row) else 'unknown'
                                manifest_data.append({'set_id': set_id, 'ver_id': ver_id, 'status': status})
                                existing[(set_id, ver_id)] = status
                            except ValueError:
                                # Skip rows where set_id or ver_id cannot be converted to int
                                continue
                except ValueError:
                    print(f"Manifest file at {MANIFEST} does not contain 'set_id' or 'ver_id' columns in header. Attempting to infer from first two columns.")
                    # Fallback if header is missing or columns not found by name
                    f.seek(0) # Reset file pointer
                    reader = csv.reader(f)
                    for row in reader:
                         if len(row) >= 2:
                            try:
                                set_id = int(row[0])
                                ver_id = int(row[1])
                                status = row[3] if len(row) > 3 else 'unknown' # Assume status is 4th column if exists
                                manifest_data.append({'set_id': set_id, 'ver_id': ver_id, 'status': status})
                                existing[(set_id, ver_id)] = status
                            except ValueError:
                                # Skip rows where set_id or ver_id cannot be converted to int
                                continue

        if manifest_data:
            manifest_df = pd.DataFrame(manifest_data)
            initial_train_df = manifest_df[['set_id', 'ver_id']].astype({'set_id':int,'ver_id':int})
            print(f"Loaded {len(initial_train_df)} entries from {MANIFEST} to use as initial training tracks.")
        else:
            print(f"No valid entries found in manifest file at {MANIFEST}. Starting with an empty initial training set.")

    except Exception as e:
        print(f"Error processing manifest file at {MANIFEST}: {e}. Starting with an empty initial training set.")
        initial_train_df = pd.DataFrame(columns=['set_id', 'ver_id'])
        existing = {} # Reset existing if loading fails

else:
    print(f"Manifest file not found at {MANIFEST}. Starting with an empty initial training set.")

#target split size
n_train_total = 2000
n_val = 150
n_test = 150

#training subset
# Ensure initial training tracks are within the original training split
initial_train_in_original_train = initial_train_df.merge(original_train_df, on=['set_id','ver_id'], how='inner')
n_initial_train = len(initial_train_in_original_train)

if n_initial_train >= n_train_total:
    print(f" Initial training list already exceeds training target ({n_initial_train} >= {n_train_total}). Using only initial entries.")
    train_sel = initial_train_in_original_train.sample(n=n_train_total, random_state=42)
else:
    remaining_needed = n_train_total - n_initial_train
    # Exclude already included ones from the original training split
    remaining_train = original_train_df.merge(initial_train_in_original_train, on=['set_id','ver_id'], how='left', indicator=True)
    remaining_train = remaining_train[remaining_train['_merge'] == 'left_only'].drop(columns='_merge')
    # Randomly sample remainder from the original training split
    random_extra = remaining_train.sample(n=min(remaining_needed, len(remaining_train)), random_state=42)
    train_sel = pd.concat([initial_train_in_original_train, random_extra], ignore_index=True)

print(f"Training subset size: {len(train_sel)} ({n_initial_train} initial + {len(train_sel) - n_initial_train} random)")

# val and test splits
# Ensure validation and test tracks have at least one version in the training set
train_set_ids = set(train_sel['set_id'])

# Filter original validation and test splits to include only sets present in the training set
eligible_val_df = original_val_df[original_val_df['set_id'].isin(train_set_ids)]
eligible_test_df = original_test_df[original_test_df['set_id'].isin(train_set_ids)]

# Sample from the eligible validation and test sets
val_sel  = eligible_val_df.sample(n=min(n_val, len(eligible_val_df)), random_state=42)
test_sel = eligible_test_df.sample(n=min(n_test, len(eligible_test_df)), random_state=42)

# Combine all selected tracks (train, val, test)
all_selected_tracks = pd.concat([train_sel, val_sel, test_sel], ignore_index=True).drop_duplicates(subset=['set_id', 'ver_id'])

# Merge with list_df to get URLs for all selected tracks
download_df = all_selected_tracks.merge(list_df[['set_id','ver_id','url']], on=['set_id','ver_id'], how='left')

# Drop rows without a URL
download_df = download_df.dropna(subset=['url'])

print(f"Training subset: {len(train_sel)} total ({n_initial_train} initial + {len(train_sel) - n_initial_train} random)")
print(f"Validation subset: {len(val_sel)}")
print(f"Test subset: {len(test_sel)}")
print(f"Total tracks selected for download with valid URLs: {len(download_df)}")
display(download_df.head())

#show how many tracks already exist and are downloaded
print(f"\n Existing manifest entries for resuming downloads: {len(existing)}")

Audio will be stored in: /content/drive/MyDrive/CQTNet/crawl_data/data
Manifest file for resuming: /content/drive/MyDrive/CQTNet/crawl_data/data/download_manifest.csv
Loaded 1328 entries from /content/drive/MyDrive/CQTNet/crawl_data/data/download_manifest.csv to use as initial training tracks.
Training subset size: 2000 (1328 initial + 672 random)
Training subset: 2000 total (1328 initial + 672 random)
Validation subset: 0
Test subset: 0
Total tracks selected for download with valid URLs: 1499


,set_id,ver_id,url
0,188,3,https://www.youtube.com/watch?v=z96m4jzZ2nQ
1,1612,8,https://www.youtube.com/watch?v=B67GrQqoho4
2,1156,4,https://www.youtube.com/watch?v=8rU6AT-AK9w
3,1349,24,https://www.youtube.com/watch?v=DKXM4ldT6SM
4,23,82,https://www.youtube.com/watch?v=_Tc9CBlbumM


Existing manifest entries for resuming downloads: 827


# download files

## i encountered many many many failed downloads form yt-dlp, roughly ~40% of my original splits downloaded so i could no longer guarantee that every track in the test/val set has a version present in the training set i had start new. I identified the songs with >= 2 version already downloaded and narrowed my classes from ~8,500 to 42 songs and manually ran yt-dlp to fill out the classes to ensure they had 4-8 tracks in them. I figured it was better to have fweer classes with an okay amount of data per song.

In [ ]:
import os
import subprocess, time
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import csv
import pandas as pd # Import pandas to work with list_df

WORKERS = 4
RETRIES = 1
COOKIES = '/content/yt_cookies.txt' #cookies to get around bot detection from yt
YTDLP_BIN = 'yt-dlp'
#list of set IDs that need to be filled out
requested_set_ids = [156, 84, 475, 66, 5, 405, 1621, 240, 223, 1894, 1288, 596, 5024, 227, 582, 1138, 218, 139, 215, 209, 3871, 1174, 5453, 159, 443, 345, 3341, 2253, 294, 12, 390, 1, 712, 293, 19, 44, 964, 9, 4183, 2028, 863]

# Load the list file to get available versions
list_df = pd.read_csv('/content/list', sep='\t', header=None, engine='python', quoting=csv.QUOTE_NONE)
# Assuming the first 6 columns based on previous successful loading
if list_df.shape[1] >= 6:
    list_df.columns = ['set_id','ver_id','title','performer','url','status']
else:
    # Fallback for older versions of the list file
    if list_df.shape[1] >= 5:
         list_df.columns = ['set_id','ver_id','title','performer','url'][:list_df.shape[1]]
    else:
        if list_df.shape[1] >= 4:
             list_df.columns = ['set_id','ver_id','title','performer'][:list_df.shape[1]]
        else:
            raise ValueError(f"The list file at /content/list does not contain enough columns. Expected at least 4, found {list_df.shape[1]}.")

list_df = list_df.astype({'set_id':int,'ver_id':int})


# Load the manifest to identify existing tracks
def _read_manifest(path):
    read_attempts = []
    # Try reading with pandas first, as it's generally more robust
    try:
        df = pd.read_csv(path, engine='python', on_bad_lines='skip') # Use on_bad_lines='skip' to handle malformed rows
        if df.shape[1] >= 2:
            # Attempt to infer the separator if not comma
            if df.columns.tolist()[0] != 'set_id' and ',' not in df.columns.tolist()[0]:
                 for sep in ['\t', ';', '|']:
                      try:
                          df_alt = pd.read_csv(path, sep=sep, engine='python', on_bad_lines='skip')
                          if df_alt.shape[1] >= 2:
                               return df_alt, sep
                      except Exception as e:
                           read_attempts.append((sep, str(e)))

            return df, ',' # Assume comma if pandas read successfully or fallback didn't work
    except Exception as e:
        read_attempts.append(('pandas_initial', str(e)))


    # Fallback to csv module if pandas fails
    try:
        manifest_data = []
        with open(path, 'r', newline='') as f:
            reader = csv.reader(f)
            # Try to find header
            try:
                header = next(reader)
                header_lower = [h.lower() for h in header]
                try:
                    set_id_idx = header_lower.index('set_id')
                    ver_id_idx = header_lower.index('ver_id')
                    status_idx = -1
                    try:
                        status_idx = header_lower.index('status')
                    except ValueError:
                        pass

                    for row in reader:
                         if len(row) > max(set_id_idx, ver_id_idx):
                            try:
                                set_id = int(row[set_id_idx].strip())
                                ver_id = int(row[ver_id_idx].strip())
                                status = row[status_idx].strip() if status_idx != -1 and status_idx < len(row) else 'unknown'
                                manifest_data.append({'set_id': set_id, 'ver_id': ver_id, 'status': status})
                            except ValueError:
                                continue # Skip rows with non-integer set_id or ver_id
                except ValueError:
                     print(f"Manifest file at {path} does not contain 'set_id' or 'ver_id' columns in header. Attempting to infer from first two columns.")
                     f.seek(0) # Reset file pointer
                     reader = csv.reader(f)
                     next(reader) # Skip header again
                     for row in reader:
                         if len(row) >= 2:
                            try:
                                set_id = int(row[0].strip())
                                ver_id = int(row[1].strip())
                                status = row[3].strip() if len(row) > 3 else 'unknown'
                                manifest_data.append({'set_id': set_id, 'ver_id': ver_id, 'status': status})
                            except ValueError:
                                continue

            except StopIteration:
                print(f"Manifest file at {path} is empty or has no header.")
                header = [] # No header found

        if manifest_data:
            return pd.DataFrame(manifest_data), 'csv_module'
        else:
             raise RuntimeError(f"No valid entries found in manifest file at {path}.")

    except Exception as e:
        read_attempts.append(('csv_module', str(e)))
        raise RuntimeError(f"Failed to read manifest using common delimiters. Attempts: {read_attempts}")


manifest_path = '/content/drive/MyDrive/CQTNet/crawl_data/data/download_manifest.csv'
audio_dir = '/content/drive/MyDrive/CQTNet/crawl_data/data'

# Ensure manifest file exists with header if it's new
if not os.path.exists(manifest_path):
    os.makedirs(os.path.dirname(manifest_path), exist_ok=True)
    with open(manifest_path, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['set_id', 'ver_id', 'url', 'status', 'path', 'size', 'ext', 'error_msg', 'timestamp'])

manifest_df, used_sep = _read_manifest(manifest_path)


# Identify existing tracks from the manifest
accepted = {'exists', 'downloaded', 'present', 'local', 'available', 'ok', 'done'}

# Check if 'status' column exists before trying to use it, otherwise assume it's the 4th column based on previous logic
status_col_candidates = ['status', 'state']
status_col = None
manifest_cols_lower = [c.lower() for c in manifest_df.columns]
for cand in status_col_candidates:
    if cand in manifest_cols_lower:
        status_col = manifest_df.columns[manifest_cols_lower.index(cand)]
        break

if status_col is None and manifest_df.shape[1] >= 4:
    status_col = manifest_df.columns[3] # Fallback to 4th column

existing_tracks = set()
if status_col and not manifest_df.empty:
     manifest_df['__status_norm'] = manifest_df[status_col].astype(str).str.strip().str.lower()
     existing_mask = manifest_df['__status_norm'].apply(lambda s: any(k in s for k in accepted))
     # Need to handle potential missing set_id/ver_id columns robustly if header is missing
     try:
         set_id_col_manifest = manifest_df.columns[[c.lower() for c in manifest_df.columns].index('set_id')]
         ver_id_col_manifest = manifest_df.columns[[c.lower() for c in manifest_df.columns].index('ver_id')]
         for index, row in manifest_df[existing_mask].iterrows():
             try:
                 set_id = int(float(row[set_id_col_manifest]))
                 ver_id = int(float(row[ver_id_col_manifest]))
                 existing_tracks.add((set_id, ver_id))
             except ValueError:
                 continue # Skip rows with non-integer set_id or ver_id
     except ValueError:
         print("Warning: Could not find 'set_id' or 'ver_id' columns in manifest for filtering existing tracks.")
         # If columns not found by name, attempt using first two columns as a last resort
         if manifest_df.shape[1] >= 2:
              for index, row in manifest_df[existing_mask].iterrows():
                 try:
                     set_id = int(float(row[0]))
                     ver_id = int(float(row[1]))
                     existing_tracks.add((set_id, ver_id))
                 except ValueError:
                     continue # Skip rows with non-integer set_id or ver_id


def download_one(set_id, ver_id, url, manifest_path, audio_dir):
    out_dir = os.path.join(audio_dir, str(set_id))
    os.makedirs(out_dir, exist_ok=True)
    out_tmpl = os.path.join(out_dir, f"{ver_id}.%(ext)s")

    # Check if the expected output file already exists and is not empty
    expected_mp3_path = os.path.join(out_dir, f"{ver_id}.mp3")
    if os.path.exists(expected_mp3_path) and os.path.getsize(expected_mp3_path) > 0:
        # Check manifest status as well for a more robust 'exists' check
        if (set_id, ver_id) in existing_tracks:
            return (set_id, ver_id, url, 'exists', expected_mp3_path, os.path.getsize(expected_mp3_path), 'mp3', 'File already exists and is in manifest.')
        else:
             # If file exists but not in manifest, assume it was downloaded previously
             # Add to manifest and treat as exists
             status = 'exists'
             downloaded_path = expected_mp3_path
             downloaded_size = os.path.getsize(expected_mp3_path)
             downloaded_ext = 'mp3'
             last_err = 'File already exists.'
             ts = time.strftime('%Y-%m-%d %H:%M:%S')
             try:
                 with open(manifest_path, 'a', newline='') as f:
                     writer = csv.writer(f)
                     writer.writerow([set_id, ver_id, url, status, downloaded_path, downloaded_size, downloaded_ext, last_err, ts])
             except Exception as e:
                 print(f"[ERROR] Failed to write to manifest for {set_id}/{ver_id}: {e}")
             existing_tracks.add((set_id, ver_id)) # Add to in-memory set
             return (set_id, ver_id, url, status, downloaded_path, downloaded_size, downloaded_ext, last_err)


#build command
    cmd = [
        YTDLP_BIN,
        '-x',                # Extract audio
        '--audio-format', 'mp3', # Convert to mp3
        '--audio-quality', '0', # Highest quality
        '--no-overwrites',   # Do not overwrite existing files
        '--continue',        # Continue where left off
        '--sleep-interval', '5', # Sleep between downloads to delay being bit-limited by yt
        '-o', out_tmpl,
        url
    ]

    if os.path.exists(COOKIES):
        cmd.extend(['--cookies', COOKIES])


    last_err = ''
    status = 'failed' # Default status
    downloaded_path = ''
    downloaded_size = 0
    downloaded_ext = ''

    for attempt in range(RETRIES+1):
        try:
            proc = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, timeout=900) # Increased timeout
            out = proc.stdout.decode('utf-8', 'ignore')
            err = proc.stderr.decode('utf-8', 'ignore')

            # Check for successful download based on presence of the expected output file and non-zero size
            if os.path.exists(expected_mp3_path) and os.path.getsize(expected_mp3_path) > 0:
                status = 'downloaded'
                downloaded_path = expected_mp3_path
                downloaded_size = os.path.getsize(expected_mp3_path)
                downloaded_ext = 'mp3'
                last_err = err[:1000] # Capture stderr even on success for potential warnings
                existing_tracks.add((set_id, ver_id)) # Add to in-memory set
                break # Exit retry loop on success
            else:
                # If yt-dlp exited nonzero or no file, record stderr and prepare retry
                last_err = f"yt-dlp did not produce file. returncode={proc.returncode}. stdout (truncated): {out[:500]}. stderr (truncated): {err[:500]}"
                # Also include stderr if available for failure reasons
                if err:
                    last_err += f". stderr (truncated): {err[:500]}"


        except Exception as e:
            last_err = f"exception: {e}"

        # small backoff between attempts
        if attempt < RETRIES:
             time.sleep(5 + attempt*attempt*5) # Increased backoff


    ts = time.strftime('%Y-%m-%d %H:%M:%S')
    # Append status to manifest file immediately
    try:
        with open(manifest_path, 'a', newline='') as f:
            writer = csv.writer(f)
            writer.writerow([set_id, ver_id, url, status, downloaded_path, downloaded_size, downloaded_ext, last_err, ts])
    except Exception as e:
        print(f"[ERROR] Failed to write to manifest for {set_id}/{ver_id}: {e}")


    return (set_id, ver_id, url, status, downloaded_path, downloaded_size, downloaded_ext, last_err)


# download loop
target_versions_per_set = 5
downloads_needed = {set_id: target_versions_per_set for set_id in requested_set_ids}
tracks_to_process = []

# Prepare a list of all potential tracks for the requested set IDs, sorted by version
all_potential_tracks = []
for set_id in requested_set_ids:
    available_versions_df = list_df[list_df['set_id'] == set_id].copy()
    available_versions_df = available_versions_df.sort_values('ver_id')
    for index, row in available_versions_df.iterrows():
        all_potential_tracks.append({'set_id': row['set_id'], 'ver_id': row['ver_id'], 'url': row['url']})

print(f"Attempting to download up to {target_versions_per_set} additional tracks per set for {len(requested_set_ids)} sets.")
print(f"Total potential tracks for requested sets: {len(all_potential_tracks)}")

successful_downloads = 0
failed_attempts = 0
processed_count = 0 # To track progress through the potential tracks


# Dictionary to keep track of which tracks (set_id, ver_id) are being processed or have been attempted
processing_attempts = {}

# Filter out already existing tracks from the potential list
all_potential_tracks = [
    track for track in all_potential_tracks
    if (track['set_id'], track['ver_id']) not in existing_tracks
]

print(f"Potential tracks to attempt (excluding existing): {len(all_potential_tracks)}")

# Use a set to keep track of which set_ids have reached their target
sets_completed = {set_id for set_id, count in downloads_needed.items() if count <= 0}

# Create a list of tasks to submit to the thread pool executor
tasks_to_submit = []
for track in all_potential_tracks:
     set_id = track['set_id']
     ver_id = track['ver_id']
     if set_id in downloads_needed and downloads_needed[set_id] > 0:
          tasks_to_submit.append(track)

print(f"Tasks to be submitted for download: {len(tasks_to_submit)}")

# Use tqdm to show progress over the number of tasks submitted
with ThreadPoolExecutor(max_workers=WORKERS) as ex:
    futures = {ex.submit(download_one, t['set_id'], t['ver_id'], t['url'], manifest_path, audio_dir): t for t in tasks_to_submit}

    for fut in tqdm(as_completed(futures), total=len(futures), desc="Downloading tracks"):
        processed_count += 1 # Increment processed count for the progress bar

        res = fut.result()
        set_id, ver_id, url, status, path, size, ext, err = res

        if status == 'downloaded':
            successful_downloads += 1
            print(f"\n[SUCCESS] Downloaded: set={set_id} ver={ver_id}")
            # Decrement needed count for this set_id if it hasn't reached the target yet
            if set_id in downloads_needed and downloads_needed[set_id] > 0:
                 downloads_needed[set_id] -= 1
                 if downloads_needed[set_id] <= 0:
                     sets_completed.add(set_id) # Mark set as completed
        elif status == 'failed':
            failed_attempts += 1
            print(f"\n[FAILED] set={set_id} ver={ver_id}. Reason: {err.splitlines()[0] if err else 'Unknown error'}") # Print only first line of error
        elif status == 'exists':
             # If it exists, and we still need downloads for this set, decrement needed count
             if set_id in downloads_needed and set_id not in sets_completed:
                  downloads_needed[set_id] -= 1
                  if downloads_needed[set_id] <= 0:
                     sets_completed.add(set_id) # Mark set as completed
             # Print less verbose message for existing tracks during this download run
             # print(f"\n[INFO] Track exists: set={set_id} ver={ver_id}")


# Report final counts
print("\n---- DOWNLOAD SUMMARY ----")
print(f"Total successful downloads: {successful_downloads}")
print(f"Total failed attempts: {failed_attempts}")
print(f"Sets that reached the target of {target_versions_per_set} downloads: {len(sets_completed)}/{len(requested_set_ids)}")
print("All download attempts completed. Manifest updated with statuses.")

Attempting to download up to 5 additional tracks per set for 41 sets.
Total potential tracks for requested sets: 2744
Potential tracks to attempt (excluding existing): 2657
Tasks to be submitted for download: 2657



[FAILED] set=156 ver=1. Reason: yt-dlp did not produce file. returncode=1. stdout (truncated): [youtube] Extracting URL: https://www.youtube.com/watch?v=8llUWm8v0rc



[FAILED] set=156 ver=2. Reason: yt-dlp did not produce file. returncode=1. stdout (truncated): [youtube] Extracting URL: https://www.youtube.com/watch?v=voIFvqIREyw

[FAILED] set=156 ver=3. Reason: yt-dlp did not produce file. returncode=1. stdout (truncated): [youtube] Extracting URL: https://www.youtube.com/watch?v=xmZ_4AkXcgg



[FAILED] set=156 ver=0. Reason: yt-dlp did not produce file. returncode=1. stdout (truncated): [youtube] Extracting URL: https://www.youtube.com/watch?v=UaSMROk-D-A



[FAILED] set=156 ver=4. Reason: yt-dlp did not produce file. returncode=1. stdout (truncated): [youtube] Extracting URL: https://www.youtube.com/watch?v=G6tNZZf5kUM


...


[FAILED] set=5 ver=185. Reason: yt-dlp did not produce file. returncode=1. stdout (truncated): [youtube] Extracting URL: https://www.youtube.com/watch?v=nKVRkGS4ud8



[FAILED] set=5 ver=186. Reason: yt-dlp did not produce file. returncode=1. stdout (truncated): [youtube] Extracting URL: https://www.youtube.com/watch?v=K7TskdEcaYg



[FAILED] set=5 ver=187. Reason: yt-dlp did not produce file. returncode=1. stdout (truncated): [youtube] Extracting URL: https://www.youtube.com/watch?v=0dFYIz6u7yM



[FAILED] set=5 ver=189. Reason: yt-dlp did not produce file. returncode=1. stdout (truncated): [youtube] Extracting URL: https://www.youtube.com/watch?v=IP4kgaPtEmM


KeyboardInterrupt: 

In [ ]:
import os

# 1. Define the base directory
audio_data_path = '/content/drive/MyDrive/CQTNet/crawl_data/CQTNet_data'

# 2. Define a list of audio file extensions
audio_extensions = ['.mp3', '.wav', '.ogg', '.flac', '.aac', '.m4a']

# 7. Store audio file paths and set IDs
audio_files = []

# 3. Walk through the base directory and its subdirectories
for root, dirs, files in os.walk(audio_data_path):
    # 6. Extract the set ID from the subdirectory name
    set_id = os.path.basename(root)

    for file in files:
        # 4. Check if the file extension is in the list of audio extensions
        if any(file.lower().endswith(ext) for ext in audio_extensions):
            # 5. Construct the full path
            file_path = os.path.join(root, file)
            audio_files.append((set_id, file_path))

# Print the number of audio files found and the first few entries
print(f"Found {len(audio_files)} audio files.")
if audio_files:
    print("First 5 audio files:")
    for i in range(min(5, len(audio_files))):
        print(audio_files[i])

Found 232 audio files.
First 5 audio files:
('1288', '/content/drive/MyDrive/CQTNet/crawl_data/CQTNet_data/1288/13.mp3')
('1288', '/content/drive/MyDrive/CQTNet/crawl_data/CQTNet_data/1288/1.mp3')
('1288', '/content/drive/MyDrive/CQTNet/crawl_data/CQTNet_data/1288/3.mp3')
('1288', '/content/drive/MyDrive/CQTNet/crawl_data/CQTNet_data/1288/0.mp3')
('1288', '/content/drive/MyDrive/CQTNet/crawl_data/CQTNet_data/1288/2.mp3')


In [ ]:
# 1. Create an empty dictionary called audio_files_by_set
audio_files_by_set = {}

# 2. Iterate through the audio_files list
for set_id, file_path in audio_files:
    # 3. Use the set_id as the key and 4. Append the file_path to the list
    if set_id not in audio_files_by_set:
        audio_files_by_set[set_id] = []
    audio_files_by_set[set_id].append(file_path)

# 5. Print the number of unique set IDs found
print(f"Found {len(audio_files_by_set)} unique set IDs.")

# 6. Print the number of files associated with the first few set IDs
print("Number of files per set ID (first few):")
for i, (set_id, file_paths) in enumerate(audio_files_by_set.items()):
    if i >= 5:
        break
    print(f"Set ID: {set_id}, Number of files: {len(file_paths)}")

Found 42 unique set IDs.
Number of files per set ID (first few):
Set ID: 1288, Number of files: 5
Set ID: 5024, Number of files: 5
Set ID: 294, Number of files: 5
Set ID: 1894, Number of files: 4
Set ID: 224, Number of files: 5


In [ ]:
import random

# 1. Initialize two empty lists
train_tracks = []
remaining_tracks = []

# 2. Iterate through the audio_files_by_set dictionary
for set_id, file_paths in audio_files_by_set.items():
    # 3. Check the number of files available for each set
    num_files = len(file_paths)

    # 4. If a set has 2 or more files, randomly select 2 for training
    if num_files >= 2:
        selected_for_train = random.sample(file_paths, 2)
        train_tracks.extend(selected_for_train)
        # Add the rest to remaining tracks
        remaining_files = [f for f in file_paths if f not in selected_for_train]
        remaining_tracks.extend(remaining_files)
    else:
        # 5. If a set has fewer than 2 files, add all to training
        train_tracks.extend(file_paths)

print(f"Number of tracks initially selected for training: {len(train_tracks)}")
print(f"Number of tracks remaining for test selection: {len(remaining_tracks)}")

Number of tracks initially selected for training: 84
Number of tracks remaining for test selection: 148


In [ ]:
import os
import shutil

# Define base directory and new split directories
base_data_dir = '/content/drive/MyDrive/CQTNet/crawl_data/CQTNet_data'
train_dir = os.path.join(base_data_dir, 'train')
test_dir = os.path.join(base_data_dir, 'test')

# Create the new train and test directories if they don't exist
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Define the paths to the existing track list files
train_list_path = 'train_tracks.txt'
test_list_path = 'test_tracks.txt'

# Function to read track paths from a file
def read_track_list(file_path):
    with open(file_path, 'r') as f:
        return [line.strip() for line in f if line.strip()]

# Read the track lists
train_tracks = read_track_list(train_list_path)
test_tracks = read_track_list(test_list_path)

# Function to move and rename files
def move_and_rename_tracks(track_list, destination_dir):
    new_track_list = []
    for old_path in track_list:
        if os.path.exists(old_path):
            # Extract set ID, version ID, and extension from the old path
            # Assuming the path format is .../<set_id>/<version_id>.<format>
            try:
                set_id = os.path.basename(os.path.dirname(old_path))
                file_name = os.path.basename(old_path)
                version_id, file_extension = os.path.splitext(file_name)

                # Create the new file name and path
                new_file_name = f"{set_id}_{version_id}{file_extension}"
                new_path = os.path.join(destination_dir, new_file_name)

                # Move and rename the file
                shutil.move(old_path, new_path)
                new_track_list.append(new_path)
                # print(f"Moved and renamed: {old_path} -> {new_path}") # Optional: uncomment for verbose output

            except Exception as e:
                print(f"Error moving or renaming {old_path}: {e}")
                # If move/rename fails, keep the old path in the new list for consistency
                new_track_list.append(old_path)
        else:
            print(f"Warning: Source file not found, skipping: {old_path}")
            # If source file not found, don't include it in the new list
            pass # Or new_track_list.append(old_path) if you want to keep missing files in the list

    return new_track_list

print("Moving and renaming training files...")
new_train_tracks = move_and_rename_tracks(train_tracks, train_dir)

print("\nMoving and renaming test files...")
new_test_tracks = move_and_rename_tracks(test_tracks, test_dir)

print("\nMove and rename operations completed.")

Moving and renaming training files...

Moving and renaming test files...

Move and rename operations completed.


### i manually downloaded the tracks from here so now i just divided the data into a training and test split. no validation set since we dont have time to play around with the hyperparameters too much and we have too little data anyway. i just divided the 232 tracks into 40 test and 192 training and ensured >=2 of each class is present in the training set

In [ ]:
# Function to write track paths to a file
def write_track_list(file_path, track_list):
    with open(file_path, 'w') as f:
        for track_path in track_list:
            f.write(track_path + '\n')

print("Updating track list files...")
write_track_list('train_tracks.txt', new_train_tracks)
write_track_list('test_tracks.txt', new_test_tracks)
print("Track list files updated.")

Updating track list files...
Track list files updated.


In [ ]:
import os

# Summarize the split based on the new file locations

# Determine unique set IDs in the new training set paths
new_train_set_ids = set()
for track_path in new_train_tracks:
    try:
        # Extract set ID from the new file name format <setID>_<versionID>.<format>
        file_name = os.path.basename(track_path)
        set_id_str = file_name.split('_')[0]
        new_train_set_ids.add(set_id_str)
    except Exception as e:
        print(f"Could not extract set ID from {track_path}: {e}")

# Determine unique set IDs in the new test set paths
new_test_set_ids = set()
for track_path in new_test_tracks:
    try:
        # Extract set ID from the new file name format <setID>_<versionID>.<format>
        file_name = os.path.basename(track_path)
        set_id_str = file_name.split('_')[0]
        new_test_set_ids.add(set_id_str)
    except Exception as e:
        print(f"Could not extract set ID from {track_path}: {e}")

print(f"Total number of tracks in the training set: {len(new_train_tracks)}")
print(f"Total number of tracks in the test set: {len(new_test_tracks)}")
print(f"Number of unique set IDs in the training set: {len(new_train_set_ids)}")
print(f"Number of unique set IDs in the test set: {len(new_test_set_ids)}")

print("\nSplitting, moving, and renaming process completed.")


---- FINAL SPLIT SUMMARY ----
Total number of tracks in the training set: 192
Total number of tracks in the test set: 40
Number of unique set IDs in the training set: 42
Number of unique set IDs in the test set: 31

Splitting, moving, and renaming process completed.


In [ ]:
# 3. If there are at least 40 remaining_tracks, randomly sample 40 file paths from remaining_tracks
if len(remaining_tracks) >= 40:
    test_tracks = random.sample(remaining_tracks, 40)
else:
    # 4. If there are fewer than 40 remaining_tracks, assign all remaining_tracks to the test_tracks list
    test_tracks = remaining_tracks

# 5. Print the number of tracks selected for the test set
print(f"Number of tracks selected for the test set: {len(test_tracks)}")

Number of tracks selected for the test set: 40


In [ ]:
# 1. Create a set of the file paths that are already in the train_tracks list.
train_paths_set = set(train_tracks)

# 2. Create a set of the file paths that are in the test_tracks list.
test_paths_set = set(test_tracks)

# 3. Iterate through the remaining_tracks list.
# 4. For each track in remaining_tracks, check if its file path is present in either the training set or the test set using the sets created in steps 1 and 2.
# 5. If a track's file path is not found in either the training or test sets, append it to the train_tracks list.
for track_path in remaining_tracks:
    if track_path not in train_paths_set and track_path not in test_paths_set:
        train_tracks.append(track_path)

# 6. Print the updated number of tracks in the training set to verify the assignment.
print(f"Updated number of tracks in the training set: {len(train_tracks)}")

Updated number of tracks in the training set: 192


In [ ]:
# 1. Define the file paths for the training and test set output files.
train_output_path = 'train_tracks.txt'
test_output_path = 'test_tracks.txt'

# 2. Open the training file path in write mode ('w').
with open(train_output_path, 'w') as f:
    # 3. Write each file path in the train_tracks list to the training file, ensuring each path is on a new line.
    for track_path in train_tracks:
        f.write(track_path + '\n')

# 4. Close the training file. (This is automatically handled by the 'with' statement)

# 5. Open the test file path in write mode ('w').
with open(test_output_path, 'w') as f:
    # 6. Write each file path in the test_tracks list to the test file, ensuring each path is on a new line.
    for track_path in test_tracks:
        f.write(track_path + '\n')

# 7. Close the test file. (This is automatically handled by the 'with' statement)

print(f"Training track list saved to {train_output_path}")
print(f"Test track list saved to {test_output_path}")

Training track list saved to train_tracks.txt
Test track list saved to test_tracks.txt


### quick stats to see how many unique songs are in each set

In [ ]:
import os

# Print total number of tracks in training and test sets
print(f"Total number of tracks in the training set: {len(train_tracks)}")
print(f"Total number of tracks in the test set: {len(test_tracks)}")

# Determine unique set IDs in training set
train_set_ids = set(os.path.basename(os.path.dirname(track_path)) for track_path in train_tracks)
# Print the number of unique set IDs in the training set
print(f"Number of unique set IDs in the training set: {len(train_set_ids)}")

# Determine unique set IDs in test set
test_set_ids = set(os.path.basename(os.path.dirname(track_path)) for track_path in test_tracks)
# Print the number of unique set IDs in the test set
print(f"Number of unique set IDs in the test set: {len(test_set_ids)}")

Total number of tracks in the training set: 192
Total number of tracks in the test set: 40
Number of unique set IDs in the training set: 42
Number of unique set IDs in the test set: 31
